In [1]:
import pandas as pd 
import numpy as np 


In [2]:
medicare_tele_trends = pd.read_csv('data/clean/medtrend_for_sql.csv')
telemedicine_use = pd.read_csv('data/clean/telemeduse_for_sql.csv')
telemedicine_services = pd.read_csv('data/clean/tele_service_for_sql.csv')

print(medicare_tele_trends.shape)
print(telemedicine_use.shape)
print(telemedicine_services.shape)

(23685, 8)
(3292, 6)
(3874, 7)


In [3]:
print("Missing values in medicare_tele_trends:")
print(medicare_tele_trends.isnull().sum())

print("\nMissing values in telemedicine_use:")
print(telemedicine_use.isnull().sum())

print("\nMissing values in telemedicine_services:")
print(telemedicine_services.isnull().sum())


Missing values in medicare_tele_trends:
row_id                   0
Year-Quarter             0
State                    0
Bene_Sex_Desc            0
Bene_Age_Desc            0
Total_Bene_TH_Elig       0
Total_Bene_Telehealth    0
Pct_Telehealth           0
dtype: int64

Missing values in telemedicine_use:
row_id            0
Year-Quarter      0
Group             0
State             0
Subgroup          0
Value           188
dtype: int64

Missing values in telemedicine_services:
Year-Quarter                0
State                       0
Year                        0
Month                       0
TelehealthType              0
ServiceCount                0
RatePer1000Beneficiaries    0
dtype: int64


In [4]:
telemedicine_use['Value'].fillna(-1, inplace=True)

# Verify that missing values in the "Value" column are filled
print("Missing values in telemedicine_use after filling 'Value' column:")
print(telemedicine_use.isnull().sum())


Missing values in telemedicine_use after filling 'Value' column:
row_id          0
Year-Quarter    0
Group           0
State           0
Subgroup        0
Value           0
dtype: int64


/var/folders/tx/7d0rdqt930q3m1lsb2vvbfj40000gn/T/ipykernel_29197/553820924.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  telemedicine_use['Value'].fillna(-1, inplace=True)


In [5]:
telemedicine_services["ServiceCount"] = telemedicine_services["ServiceCount"].astype(str).str.replace(",", "")
telemedicine_services


,Year-Quarter,State,Year,Month,TelehealthType,ServiceCount,RatePer1000Beneficiaries
0,2018-1,Alabama,2018,1,Live audio/video,6905,5.6
1,2018-1,Alabama,2018,2,Live audio/video,4821,3.9
2,2018-1,Alabama,2018,3,Live audio/video,2269,1.8
3,2018-2,Alabama,2018,4,Live audio/video,2290,1.8
4,2018-2,Alabama,2018,5,Live audio/video,2380,1.9
...,...,...,...,...,...,...,...
3869,2022-3,Wyoming,2022,9,Live audio/video,2977,34.2
3870,2022-4,Wyoming,2022,10,Live audio/video,2987,34.2
3871,2022-4,Wyoming,2022,10,Remote patient monitoring,40,0.5
3872,2022-4,Wyoming,2022,11,Live audio/video,3247,36.7


In [6]:
# Modify "District Of Columbia" in the State column by changing "O" to lowercase "o"
medicare_tele_trends['State'] = medicare_tele_trends['State'].str.replace("Of", "of")

In [7]:
def remove_single_quotes1(df):
    df = df.applymap(lambda x: x.replace("'", "") if isinstance(x, str) else x)
    df = df.applymap(lambda x: x.replace(",", "") if isinstance(x, str) else x)


In [8]:
remove_single_quotes1(telemedicine_use)
remove_single_quotes1(telemedicine_services)
remove_single_quotes1(medicare_tele_trends)

/var/folders/tx/7d0rdqt930q3m1lsb2vvbfj40000gn/T/ipykernel_29197/3740963860.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace("'", "") if isinstance(x, str) else x)
/var/folders/tx/7d0rdqt930q3m1lsb2vvbfj40000gn/T/ipykernel_29197/3740963860.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(",", "") if isinstance(x, str) else x)


In [9]:

# Modify "District Of Columbia" in the State column by changing "O" to lowercase "o"
telemedicine_use['Subgroup'] = telemedicine_use['Subgroup'].str.replace("'s", "")


In [10]:
#define insert function for patients

def generate_insert_patients(df, table_name="Patients"):

    # Rename columns to match the SQL schema
    df.rename(columns={
        "Year-Quarter": "year_quarter",
        "Group": "group_category",
        "State": "state_name",
        "Subgroup": "subgroup",
        "Value": "value"
    }, inplace=True)

    # Generate INSERT statements
    insert_statements = []
    for _, row in df.iterrows():
        statement = f"""INSERT INTO {table_name} (row_id, year_quarter, state_name, group_category, subgroup, value)
        VALUES ({row['row_id']}, '{row['year_quarter']}', '{row['state_name']}', '{row['group_category']}', '{row['subgroup']}', {row['value']});"""
        insert_statements.append(statement)

    return insert_statements

In [11]:
#write insert sql for patients
sql_statements = generate_insert_patients(telemedicine_use)

# Define output path
output_path = "result/insert_patients.sql"

# Write the SQL statements to a file
with open(output_path, "w") as f:
    for stmt in sql_statements:
        f.write(stmt + "\n")

print(f"SQL file successfully saved at: {output_path}")

SQL file successfully saved at: result/insert_patients.sql


In [12]:

def generate_insert_bene(df, table_name="Beneficiary"):
    # Rename columns to match the SQL schema
    df.rename(columns={
        "Year-Quarter": "year_quarter",
        "State": "state_name",
        "Bene_Sex_Desc": "sex",
        "Bene_Age_Desc": "age_group",
        "Total_Bene_TH_Elig": "total_bene_th_elig",
        "Total_Bene_Telehealth": "total_bene_telehealth",
        "Pct_Telehealth": "pct_telehealth"
    }, inplace=True)

    # Generate INSERT statements
    insert_statements = []
    for _, row in df.iterrows():
        statement = f"""INSERT INTO {table_name} (row_id, year_quarter, state_name, sex, age_group, total_bene_th_elig, total_bene_telehealth, pct_telehealth)
        VALUES ({row['row_id']}, '{row['year_quarter']}', '{row['state_name']}', '{row['sex']}', '{row['age_group']}', {row['total_bene_th_elig']}, {row['total_bene_telehealth']}, {row['pct_telehealth']});"""
        insert_statements.append(statement)

    return insert_statements


In [13]:


# Generate SQL statements
sql_statements = generate_insert_bene(medicare_tele_trends)

# Define output path
output_path = "result/insert_beneficiary.sql"

# Write the SQL statements to a file
with open(output_path, "w") as f:
    for stmt in sql_statements:
        f.write(stmt + "\n")

print(f"SQL file successfully saved at: {output_path}")

SQL file successfully saved at: result/insert_beneficiary.sql


In [14]:

def generate_insert_serc(df, table_name="ServiceCount"):
    # Rename columns to match the SQL schema
    df.rename(columns={
        "Year-Quarter": "year_quarter",
        "State": "state_name",
        "Year": "year",
        "Month": "month",
        "TelehealthType": "service_type",
        "ServiceCount": "service_count",
        "RatePer1000Beneficiaries": "rate_per_1000_beneficiaries"
    }, inplace=True)

    # Generate INSERT statements
    insert_statements = []
    for _, row in df.iterrows():
        statement = f"""INSERT INTO {table_name} (year_quarter, state_name, year, month, service_type, service_count, rate_per_1000_beneficiaries)
        VALUES ('{row['year_quarter']}', '{row['state_name']}', {row['year']}, {row['month']}, '{row['service_type']}', {row['service_count']}, {row['rate_per_1000_beneficiaries']});"""
        insert_statements.append(statement)

    return insert_statements



In [15]:

# Generate SQL statements
sql_statements = generate_insert_serc(telemedicine_services)

# Define output path
output_path = "result/insert_service_count.sql"

# Write the SQL statements to a file
with open(output_path, "w") as f:
    for stmt in sql_statements:
        f.write(stmt + "\n")

print(f"SQL file successfully saved at: {output_path}")


SQL file successfully saved at: result/insert_service_count.sql


In [16]:
# # Display the unique states directly in the output
# print("Unique States in Medicare Tele Trends:")
# print(f"Count: {len(unique_states_medicare)}")
# print(unique_states_medicare)
# print("\n")

# print("Unique States in Telemedicine Use:")
# print(f"Count: {len(unique_states_teleuse)}")
# print(unique_states_teleuse)
# print("\n")

# print("Unique States in Telemedicine Services:")
# print(f"Count: {len(unique_states_teleservice)}")
# print(unique_states_teleservice)


In [17]:
# # Find states that are in Medicare Tele Trends but not in the other two datasets
# unique_medicare = set(unique_states_medicare)
# unique_teleuse = set(unique_states_teleuse)
# unique_teleservice = set(unique_states_teleservice)

# # Identify the states that are only in Medicare Tele Trends
# exclusive_medicare_states = unique_medicare - (unique_teleservice | unique_teleuse )
# exclusive_medicare_states = unique_teleuse - (unique_medicare | unique_teleservice )
# # Display the result
# exclusive_medicare_states


In [18]:
# us_states = {
#     "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", 
#     "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", 
#     "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", 
#     "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", 
#     "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", 
#     "New Hampshire", "New Jersey", "New Mexico", "New York", 
#     "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", 
#     "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
#     "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
#     "West Virginia", "Wisconsin", "Wyoming", "National", "United States"
# }

# print(len(us_states))  # Confirm the number of unique states


In [19]:
# unique_medicare = unique_medicare - us_states
# unique_teleservice = unique_teleservice - us_states
# unique_teleuse = unique_teleuse - us_states
# print(unique_medicare)
# print(unique_teleservice)
# print(unique_teleuse)

